In [1]:
import nibabel as nib
import numpy as np
import torch
import glob
import os

def open_nii(filepath): 
    # Load the NIfTI file
    nii_file = nib.load(filepath)
    # Get the image data array
    image_data = nii_file.get_fdata()
    # Accessing metadata
    header = nii_file.header
    return image_data


In [2]:
from resnet import resnet50

num_seg_class = 2
data_dir = "Resnet-10_pretrained"
pretrained = True

model = resnet50(
sample_input_W=32,
sample_input_H=32,
sample_input_D=32,
shortcut_type='B',
num_seg_classes=num_seg_class)
net_dict = model.state_dict()

pretrain = torch.load("MedicalNet_pytorch_files2/pretrain/resnet_50.pth")
pretrain_dict = {".".join(k.split('.')[1:]): v for k, v in pretrain['state_dict'].items() if ".".join(k.split('.')[1:]) in net_dict.keys()}

if pretrained:
    net_dict.update(pretrain_dict)
    model.load_state_dict(net_dict)
    print(len(pretrain_dict))


/data2/om/ADNI dataset/resnet.py:173: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


318


In [3]:
from dataset import BrainDataset
from torch.utils.data import DataLoader

dataset = BrainDataset("data", mode='all')  # Load all data
# Move the model and dataset to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# dataset = dataset.to(device)
dataloader = DataLoader(dataset, batch_size=2, shuffle=False)




In [4]:
def process_data(data_dir, model, dataloader, output_dim=(10, 8, 8, 8)):
    # Iterate over the DataLoader
    for data, labels in dataloader:
        outputs = model(data.to(device)).detach().cpu().numpy()  # Assuming model output is torch.Tensor and needs to be converted to numpy
        # Process or save outputs as needed, for example:
        
        for i, label in enumerate(labels):
            feature_dir = f"features/{label}"
            os.makedirs(feature_dir, exist_ok=True)
            existing_files = glob.glob(os.path.join(feature_dir, '*.npy'))
            if existing_files:
                highest_number = max([int(os.path.splitext(os.path.basename(file))[0]) for file in existing_files])
                next_number = highest_number + 1
            else:
                next_number = 1
            output_path = os.path.join(feature_dir, f"{next_number}.npy")
            np.save(output_path, outputs[i])
            print(output_path)
        
        

process_data(data_dir="data", model=model, dataloader=dataloader)


features/1/1.npy
features/1/2.npy
features/1/3.npy
features/1/4.npy
features/1/5.npy
features/1/6.npy
features/1/7.npy
features/1/8.npy
features/1/9.npy
features/1/10.npy
features/1/11.npy
features/1/12.npy
features/1/13.npy
features/1/14.npy
features/1/15.npy
features/1/16.npy
features/1/17.npy
features/1/18.npy
features/1/19.npy
features/1/20.npy
features/1/21.npy
features/1/22.npy
features/1/23.npy
features/1/24.npy
features/1/25.npy
features/1/26.npy
features/1/27.npy
features/1/28.npy
features/1/29.npy
features/1/30.npy
features/1/31.npy
features/1/32.npy
features/1/33.npy
features/1/34.npy
features/1/35.npy
features/1/36.npy
features/1/37.npy
features/1/38.npy
features/1/39.npy
features/1/40.npy
features/1/41.npy
features/1/42.npy
features/1/43.npy
features/1/44.npy
features/1/45.npy
features/1/46.npy
features/1/47.npy
features/1/48.npy
features/1/49.npy
features/1/50.npy
features/1/51.npy
features/1/52.npy
features/1/53.npy
features/1/54.npy
features/1/55.npy
features/1/56.npy
f

In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Directory containing the .npy files
data_dir = 'Resnet-50_random'

# Get a list of all .npy files in the directory
labels = []
# npy_files = [file for file in os.listdir("features/0") if file.endswith('.npy')]
npy_files = []

for file in os.listdir("features/0"):
    npy_files.append(f"features/0/{file}")
    labels.append(0)

for file in os.listdir("features/1"):
    npy_files.append(f"features/1/{file}")
    labels.append(1)

In [6]:
data_list = []
new_label = []
# Load data and labels from .npy files
for idx, npy_file in enumerate(npy_files):
    # Load data from the file
    loaded_data = np.load(npy_file)
    data_list.append(loaded_data)
    
    # Append the corresponding label for each sample in loaded_data
    label = labels[idx]
    new_label.extend([label] * loaded_data.shape[0])  # Repeat label for each sample
labels = new_label

In [8]:
data = np.vstack(data_list)
# Convert labels to an array
labels = np.array(labels)
# # Reduce perplexity value
perplexity = 1
new_data = data.reshape(2614, -1)  
# # Apply t-SNE to reduce dimensions to 2D
tsne = TSNE(n_components=2, random_state=42)
data_2d = tsne.fit_transform(new_data)

print(new_data.shape)
# # Create a list of unique labels
unique_labels = np.unique(labels)

# Plot the 2D t-SNE representation with different colors for each label
plt.figure(figsize=(10, 8))
for label in unique_labels:
    label_mask = labels == label
    print(label)
    plt.scatter(data_2d[label_mask, 0], data_2d[label_mask, 1], label=label, alpha=0.7)

plt.title('t-SNE Visualization for Different Labels')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.legend()
plt.grid(True)
plt.show()


ValueError: cannot reshape array of size 222159872 into shape (2614,newaxis)